# SETUP

In previous lecture, you already create a database and two relational tables. Now, we use them again. However, you may create the new one to ease your work by running this queries at once.

```sql
-- Create a new database (Please run this query separately with the next one)
CREATE DATABASE w2d2pm;
```


```sql
-- Create the students table
CREATE TABLE students (
    id SERIAL PRIMARY KEY,
    name VARCHAR(50),
    age INTEGER,
    campus_id INTEGER,
    total_grade FLOAT
);

-- Insert data into the students table
INSERT INTO students (name, age, campus_id, total_grade)
VALUES
    ('Rafif Iman', 20, 1, 85.5),
    ('Hana Arisona', 21, 2, 90.2),
    ('Raka Purnomo', 19, 1, 78.9),
    ('Danu Irfansyah', 20, 3, 92.7),
    ('Rachman Ardhi', 22, 2, 88.1);

-- Create the campus table
CREATE TABLE campus (
    id SERIAL PRIMARY KEY,
    campus_name VARCHAR(50),
    batch VARCHAR(10),
    start_date DATE
);

-- Insert data into the campus table
INSERT INTO campus (campus_name, batch, start_date)
VALUES
    ('Remote', 'RMT-1', '2023-01-01'),
    ('Jakarta', 'HCK-2', '2023-02-01'),
    ('BSD', 'BSD-4', '2023-03-01'),
    ('Surabaya', 'SUB-1', '2023-04-01'),
    ('Singapore', 'SIN-1', '2023-05-01');
```

# Transaction Control Language Practice


## Pre-Explanation

In PostgreSQL, a transaction is a sequence of database operations that are grouped together to be treated as a single unit. The purpose of using transactions is to ensure that all the operations within the transaction are either fully completed or fully rolled back in case of failure, maintaining the integrity and consistency of the database.

If you don't include any TCL commands (COMMIT or ROLLBACK), and you are operating in autocommit mode (the default mode), each SQL statement will be treated as a separate transaction. In this case, the changes made by each statement will be immediately committed to the database upon its execution. There is no explicit rollback or grouping of statements as a transaction.

## COMMIT

Let's doing a practice of COMMIT command


```sql
-- Begin a transaction
BEGIN;

-- Delete a student
DELETE FROM students
WHERE id = 3;

-- Insert a new campus
INSERT INTO campus (campus_name, batch, start_date)
VALUES ('London', 'LDN-1', '2023-07-01');

-- Update the batch of an existing campus
UPDATE campus
SET batch = 'LDN-2'
WHERE id = 6;

-- Commit the transaction
COMMIT;
```

- The `BEGIN;` statement starts a transaction.
- The `DELETE FROM` statement removes a student from the students table where the ID is 3.
- The `INSERT INTO` statement adds a new campus called 'London' to the campus table with the batch 'LDN-1' and a start date of '2023-07-01'.
- The `UPDATE` statement modifies the batch of an existing campus with the ID of 6 and sets it to 'LDN-2'.
- The `COMMIT;` statement permanently applies the changes made within the transaction to the database.

By using the `COMMIT` command, the changes made within the transaction (the delete, insert, and update operations) are saved permanently in the database. If any error occurs before the `COMMIT` command, the changes would be rolled back, and the database would remain unaffected.



What happen if any error occurs before COMMIT? Here's a scenario to illustrate how errors before the `COMMIT` command trigger automatic rollback within a transaction:


```sql
-- Begin a transaction
BEGIN;

-- Insert a new student with an incorrect column order
INSERT INTO students (age, name, campus_id, total_grade)
VALUES (23, 'M. Fahmi', 2, 95.0);

-- Update an invalid column name in the campus table
UPDATE campus
SET invalid_column = 'Invalid Value'
WHERE id = 1;

-- Commit the transaction
COMMIT;
```

In this example, there are intentional errors introduced in the SQL statements:

1. In the `INSERT` statement, the column order for the values is incorrect. Instead of `age, name, campus_id, total_grade`, it is specified as `age, name, total_grade, campus_id`. This will cause a column mismatch error.

2. In the `UPDATE` statement, an invalid column name, `invalid_column`, is used, which does not exist in the campus table. This will result in an error.

Since errors occur before the `COMMIT` command, the transaction will be automatically rolled back. As a result, the changes made by the `INSERT` statement and the `UPDATE` statement will be undone, and the database will remain in its original state.

This behavior ensures data integrity by preventing incomplete or erroneous data from being committed to the database.

**NOTE:**
If you get an error `ERROR: current transaction is aborted, commands ignored until end of transaction block` when you run any query because of the error when commited the transaction, you can change/create a new connection and running again the query on `SETUP` section. (This step will restart your database in the new connection).

However, if you switch to previous connection, your tables and changes doesn't gone and the error doesn't appear anymore.

## ROLLBACK

Here's an example to perform the ROLLBACK command:

```sql
-- Begin a transaction
BEGIN;

-- Delete a student
DELETE FROM students
WHERE id = 3;

-- Insert a new campus
INSERT INTO campus (campus_name, batch, start_date)
VALUES ('London', 'LDN-1', '2023-07-01');

-- Update the batch of an existing campus (with an error)
UPDATE campus
SET invalid_column = 'LDN-2'
WHERE id = 2;

-- Rollback the transaction
ROLLBACK;
```

- The `BEGIN;` statement starts a transaction, allowing a group of database operations to be treated as a single unit.
- The `DELETE FROM` statement removes a student from the students table where the ID is 3.
- The `INSERT INTO` statement adds a new campus called 'London' to the campus table with the batch 'LDN-1' and a start date of '2023-07-01'.
- The `UPDATE` statement attempts to modify the `invalid_column` (which doesn't exist) of an existing campus with the ID of 2. This introduces an intentional error to demonstrate the rollback.
- The `ROLLBACK;` statement rolls back the transaction, undoing all the changes made within the transaction and returning the database to its state before the transaction started.

By using the `ROLLBACK` command, all changes made within the transaction (the delete, insert, and update operations) are reverted, and the database remains unaffected. It ensures that any errors encountered during the transaction cause a rollback, maintaining data integrity and consistency.

In this example, the `UPDATE` statement generates an error due to the use of an invalid column name. As a result, the transaction is rolled back, and the insertion of the new campus and the deletion of the student are undone, leaving the database in its original state before the transaction began.


Here's an example of SAVEPOINT command usage:


```sql
-- Begin a transaction
BEGIN;

-- Insert a new student
INSERT INTO students (name, age, campus_id, total_grade)
VALUES ('John Doe', 18, 2, 95.0);

-- Savepoint declaration
SAVEPOINT my_savepoint;

-- Update an existing campus
UPDATE campus
SET campus_name = 'New Campus'
WHERE id = 1;

-- Rollback to savepoint
ROLLBACK TO SAVEPOINT my_savepoint;

-- Commit the transaction
COMMIT;
```

Explanation:
1. The `BEGIN;` statement starts a transaction, allowing a group of database operations to be treated as a single unit.
2. The `INSERT INTO` statement adds a new student, 'John Doe', to the students table with an age of 18, campus ID of 2, and a total grade of 95.0.
3. The `SAVEPOINT` command is used to declare a savepoint within the transaction. A savepoint allows you to mark a specific point in the transaction to which you can later rollback.
4. The `UPDATE` statement modifies an existing campus with the ID of 1 by changing its campus name to 'New Campus'. This introduces a change that we might want to rollback.
5. The `ROLLBACK TO SAVEPOINT` statement rolls back the transaction to the specified savepoint. In this case, we rollback to the `my_savepoint` savepoint, undoing the changes made after the savepoint declaration.
6. The `COMMIT;` statement is used to permanently apply the changes made within the transaction to the database. Since we have already performed a rollback to the savepoint, this commit has no effect on the database state.

By using savepoints and the `ROLLBACK TO SAVEPOINT` command, you have the flexibility to selectively rollback a transaction to a specific point within the transaction, while still having the option to commit the changes made up until that point.

In this example, after inserting a new student, we declare a savepoint called `my_savepoint`. Then, we update an existing campus. However, when we rollback to the savepoint, only the changes made after the savepoint declaration are undone, leaving the insertion of the new student intact.

Savepoints are useful when you want to have finer-grained control over the rollback behavior within a transaction. They allow you to isolate and undo specific changes while preserving the changes made prior to the savepoint.

## RELEASE

RELEASE is used to remove a specified savepoint and all subsequent savepoints within a transaction. To do it, you can run this set of queries:

```sql
-- Begin a transaction
BEGIN;

-- Insert a new student
INSERT INTO students (name, age, campus_id, total_grade)
VALUES ('John Doe', 18, 2, 95.0);

-- Savepoint declaration
SAVEPOINT my_savepoint;

-- Update an existing campus
UPDATE campus
SET campus_name = 'New Campus'
WHERE id = 1;

-- Release the savepoint
RELEASE SAVEPOINT my_savepoint;

-- Commit the transaction
COMMIT;
```


1. The `BEGIN;` statement starts a transaction, allowing a group of database operations to be treated as a single unit.
2. The `INSERT INTO` statement adds a new student, 'John Doe', to the students table with an age of 18, campus ID of 2, and a total grade of 95.0.
3. The `SAVEPOINT` command is used to declare a savepoint within the transaction. A savepoint allows you to mark a specific point in the transaction to which you can later rollback.
4. The `UPDATE` statement modifies an existing campus with the ID of 1 by changing its campus name to 'New Campus'. This introduces a change that we might want to rollback.
5. The `RELEASE SAVEPOINT` statement releases the specified savepoint. In this case, we release the `my_savepoint` savepoint, indicating that we no longer need to rollback to that savepoint.
6. The `COMMIT;` statement is used to permanently apply the changes made within the transaction to the database. Since we released the savepoint, it has no effect on the database state.

By using savepoints and the `RELEASE SAVEPOINT` command, you can release a specific savepoint within a transaction, freeing up any resources associated with it. This allows you to continue with the transaction without the option to rollback to the released savepoint.

In this example, after inserting a new student and declaring a savepoint, we update an existing campus. However, we release the savepoint, indicating that we no longer need to rollback to that point. As a result, the changes made after the savepoint declaration are preserved, and the transaction proceeds normally.

The `RELEASE SAVEPOINT` command is useful when you want to discard the option to rollback to a specific savepoint and continue the transaction without considering it as a rollback point. It provides a way to manage the granularity of rollback control within a transaction.

## COMBINE

We'll use TCL commands to control the transaction and demonstrate how to commit, rollback, set savepoints, and release them.

```
-- Begin a transaction
BEGIN;

-- Insert a new student
INSERT INTO students (name, age, campus_id, total_grade)
VALUES ('John Doe', 18, 2, 95.0);

-- Savepoint declaration
SAVEPOINT my_savepoint;

-- Update an existing campus
UPDATE campus
SET campus_name = 'New Campus'
WHERE id = 1;

-- Rollback to savepoint
ROLLBACK TO SAVEPOINT my_savepoint;

-- Insert a new campus
INSERT INTO campus (campus_name, batch, start_date)
VALUES ('London', 'LDN-1', '2023-07-01');

-- Release the savepoint
RELEASE SAVEPOINT my_savepoint;

-- Commit the transaction
COMMIT;
```


- We start the practice session by introducing the purpose of the session, which is to simulate a transaction for managing student records and campus information using TCL commands.
- The `BEGIN;` statement starts a transaction, creating a transaction block for grouping the subsequent database operations.
- The `INSERT INTO` statement adds a new student, 'John Doe', to the students table with an age of 18, campus ID of 2, and a total grade of 95.0.
- The `SAVEPOINT` command is used to declare a savepoint called `my_savepoint`. This allows us to mark a specific point within the transaction to which we can later rollback.
- The `UPDATE` statement modifies an existing campus with the ID of 1 by changing its campus name to 'New Campus'. This introduces a change that we might want to rollback.
- The `ROLLBACK TO SAVEPOINT` statement rolls back the transaction to the specified savepoint (`my_savepoint`). It undoes the changes made after the savepoint declaration, reverting the database to its state at the savepoint.
- We continue the transaction by inserting a new campus, 'London', into the campus table with the batch 'LDN-1' and a start date of '2023-07-01'.
- The `RELEASE SAVEPOINT` command releases the savepoint (`my_savepoint`), indicating that we no longer need to rollback to that point. It removes the savepoint from the transaction, freeing up any associated resources.
- Finally, the `COMMIT;` statement is used to permanently apply the changes made within the transaction to the database. It signifies the successful completion of the transaction and makes the changes visible to other users or transactions.



# Data Control Language

Data Control Language (DCL) is a subset of SQL (Structured Query Language) that is used to control access and permissions in a database management system. DCL provides the commands necessary to define and manage user access rights, permissions, and security policies within a database.

Before you practice, you may create two different users beside the administrator user or superuser (the username should be `postgres` (by default) or any username that you set when you install the PostgreSQL).

## Create Users

Before you go further, you will create two users that are `user1` and `user2` (or you can set any names that you want).

```sql
CREATE USER user1 WITH PASSWORD '1234';

CREATE USER user2 WITH PASSWORD '1234';
```




### See Existing Users List

If you want to see the existing users, you can run this query:

```sql
SELECT * FROM pg_user;
```

This query retrieves all the rows from the pg_user system catalog table, which contains information about the database users.


## Switching User

To switch to an existing user, you can perform it by this command:

```sql
SET ROLE user1;
```

Then, to switch to the administrator user or superuser in PostgreSQL within a query, you can use the `SET ROLE` command. Here's an example:

```sql
SET ROLE postgres;
```

Replace `postgres` with the username of the administrator user or superuser in your PostgreSQL database.


### Check the Current Active User

To check the current user in PostgreSQL, you can use the `current_user` function or the `SESSION_USER` keyword. Here's how you can do it:

1. Execute the following SQL query to fetch the current user using the `current_user` function:
   ```sql
   SELECT current_user;
   ```

   The `current_user` function returns the name of the current user who is executing the query.

   Example output:
   ```
   current_user
   -------------
   user1
   ```

   The output will display the name of the current user.

2. Alternatively, you can also use the `SESSION_USER` keyword to retrieve the current user within a query:
   ```sql
   SELECT SESSION_USER;
   ```

   The `SESSION_USER` keyword returns the name of the current user for the current database session.

   Example output:
   ```
   session_user
   -------------
   user1
   ```

   The output will show the name of the current user.

Both methods, `current_user` function and `SESSION_USER` keyword, provide the ability to check the current user in PostgreSQL. Use either of these methods based on your preference or the specific requirements of your query.

## GRANT

`GRANT` is used to grant specific privileges or permissions to users or roles on database objects such as tables, views, functions, or schemas.


Here is an example use case for the `GRANT` command in PostgreSQL:

Assuming we want to grant specific privileges to `user1` and `user2` on these tables:

1. Grant `SELECT` privilege on the `students` table to `user1`:
   ```sql
   GRANT SELECT ON students TO user1;
   ```

   if you want to grant `SELECT` to `user1` for all tables:

   ```sql
   GRANT SELECT ON ALL TABLES IN SCHEMA public TO user1;
   ```

   This command allows `user1` to select (read) data from the `students` table, but it does not grant any other privileges such as inserting, updating, or deleting data from the table.

2. Grant `SELECT`, `INSERT`, `UPDATE`, and `DELETE` privileges on the `campus` table to `user2`:
   ```sql
   GRANT SELECT, INSERT, UPDATE, DELETE ON campus TO user2;

   GRANT USAGE, select on sequence campus_id_seq to user2;
   ```

   This command grants `user2` the ability to select, insert, update, and delete data from the `campus` table.

   The effect of auto increment scheme on the table makes we cannot directly given access on `INSERT`, if we already grant the privilege, it still doesn't work. So, we use this query `GRANT USAGE, select on sequence campus_id_seq to user2` to grant of sequence/auto increment access. `campus_id_seq` follows format of `TableName_PrimaryKeyColumnName_seq`, for `students` table the format should be `students_id_seq`.

To check what privileges that posseses by the specific user, you can run this query:

```sql
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE table_name = 'students' AND grantee = 'user1';
```

```sql
SELECT grantee, privilege_type
FROM information_schema.table_privileges
WHERE table_name = 'campus' AND grantee = 'user2';
```

all information about users privileges store at table_privileges table in the information_schema database.

By the way, If you running `UPDATE` or `INSERT` or any command except `SELECT` on students table for user1, you still get a successful message, means that it's still possible to run those commands.

So, you really need `REVOKE` to restrict the certain commands.

## REVOKE

By using the `REVOKE` command, you can selectively remove specific privileges from users, effectively restricting their access to certain commands or operations on database objects. If you want to control and limit the actions a user can perform on a table, you would typically use a combination of `GRANT` and `REVOKE` commands to fine-tune their access.

For example, let's consider the case where you have granted the `SELECT` privilege to a user on a table. If you also want to prevent them from performing `UPDATE` or `INSERT` operations on the same table, you would need to explicitly revoke those privileges using the `REVOKE` command.

Here's an example:

1. First, create a new user namely `user3`
    ```sql
   CREATE USER user3 WITH PASSWORD '1234';
   ```

2. Then, you may grant all privilage on the `campus` table to `user3`:

  ```sql
  GRANT ALL PRIVILAGES ON campus TO user3;
  ```

  Futhermore, if you want to give all privilages to all tables to `user3` you can use:
  ```sql
  GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA MY_SCHEMA TO user3;
  ```
  
  You can change MY_SCHEMA into `public` (default schema name in PostgreSQL)

3. Grant `SELECT` privilege on the `campus` table to `user1`:
   ```sql
   GRANT SELECT ON students TO user1;
   ```

4. Test `UPDATE` and `INSERT` before revoking the privileges:

  ```sql
  UPDATE campus SET batch_name = 'RMT-5' where campus_name='Remote';
  ```

  ```sql
  INSERT INTO campus (campus_name, batch, start_date)
  VALUES ('London', 'LDN-1', '2023-09-01');
  ```

5. Revoke `UPDATE` and `INSERT` privileges on the `students` table from `user1`:
   ```sql
   REVOKE UPDATE, INSERT ON campus FROM user1;
   ```

This sequence of commands grants `SELECT` privilege to `user1`, allowing them to retrieve data from the `students` table, but revokes their ability to perform `UPDATE` and `INSERT` operations on the same table.

Finally, you can test the `REVOKE` effect:

Test `UPDATE` and `INSERT` before revoking the privileges:

  ```sql
  UPDATE campus SET campus_name = 'Pondok Indah' where id=2;
  ```

  ```sql
  INSERT INTO campus (campus_name, batch, start_date)
  VALUES ('Medan', 'MDN-1', '2023-11-01');
  ```

## DENY

In PostgreSQL, the DENY command is not available. The DENY command is part of the SQL standard, but it is not supported in PostgreSQL. Instead, PostgreSQL provides the REVOKE command to revoke privileges and restrict access to database objects.

In essence, the key difference between REVOKE and DENY is that REVOKE removes previously granted privileges, while DENY explicitly denies the acquisition of certain privileges. Both commands are used to control access and manage permissions in SQL databases, but their specific behavior may vary across different database management systems.

# COMBINING TCL and DCL

You can perform the data privilege control using transaction schema. It is possible to do that.

Here is an example how to create a transaction start with creating users (user4 and user5) along with granting and revoking access privileges.

```sql
-- Start a transaction
BEGIN;

-- Create user4
CREATE USER user4 WITH PASSWORD '1234';

-- Create user5
CREATE USER user5 WITH PASSWORD '1234';

-- Grant SELECT privilege on the students table to user4
GRANT SELECT ON students TO user4;

-- Grant INSERT and UPDATE privileges on the campus table to user5
GRANT INSERT, UPDATE ON campus TO user5;

-- Revoke INSERT privilege on the campus table from user5
REVOKE INSERT ON campus FROM user5;

-- Revoke SELECT privilege on the students table from user4
REVOKE SELECT ON students FROM user4;

-- Commit the transaction
COMMIT;
```

In this example, we begin a transaction using the `BEGIN` command. Within the transaction, we create `user4` and `user5` using the `CREATE USER` command. Then, we grant the `SELECT` privilege on the `students` table to `user4` using the `GRANT` command, and the `INSERT` and `UPDATE` privileges on the `campus` table to `user5` using the `GRANT` command as well.


Later in the transaction, we revoke the `INSERT` privilege on the `campus` table from `user5` using the `REVOKE` command, and the `SELECT` privilege on the `students` table from `user4` using the `REVOKE` command.

Finally, when you are satisfied with the changes made within the transaction, you can commit the transaction using the `COMMIT` command to make the changes permanent.